<a href="https://colab.research.google.com/github/katrinag2004/dsproject2/blob/main/cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd

# Step 1: Load the dataset
df = pd.read_csv("/content/tracks_features.csv")

# Step 2: Drop unwanted columns
columns_to_drop = [
    'speechiness', 'acousticness', 'instrumentalness', 'liveness',
    'valence', 'tempo', 'time_signature', 'disc_number', 'track_number',
    'key', 'loudness', 'danceability'
]
df = df.drop(columns=columns_to_drop)


# Step 4: Keep only relevant columns (edit these based on your dataset's actual column names)
# For example, assume the relevant columns are:
# 'id' = track ID, 'artists_id' = artist ID, 'name' = track name

columns_to_keep = ['id','artists', 'artist_ids', 'name']  # Modify as needed
df = df[columns_to_keep]

# Step 5: Drop duplicate artist entries (keeping first song per artist)
df = df.drop_duplicates(subset='artist_ids', keep='first')

# Step 6: Save the cleaned dataset
df.to_csv("modified_songs3.csv", index=False)

print("✅ Modified dataset saved with", len(df), "rows.")


✅ Modified dataset saved with 166423 rows.


In [20]:
from google.colab import files
files.download("modified_songs3.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>